## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

This first implementation will use a simple, brute-force type of RAG..

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
# imports for langchain, plotly and Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain   
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.globals import set_debug


In [3]:
set_debug(True)

In [4]:
# price is a factor for our company, so we're going to use a low cost model
EMBEDDING_MODEL = 'ibm-granite/granite-embedding-125m-english'
MODEL = "gpt-4o-mini"
GRANITE_MODEL = 'granite3.3:8b'
db_name = "surya_knowledgebase"

In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [7]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("/Users/suryan0800/Documents/Surya Developer Profession/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/[!.~]*")
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBB

Total number of chunks: 98
Document types found: {'Eminence', 'Awards', 'Jira', 'Stats and Diagrams', 'Education', 'Resume'}


In [8]:
[(ind, doc.metadata['source']) for ind, doc in enumerate(documents)]

[(0,
  '/Users/suryan0800/Documents/Surya Developer Profession/Resume/Surya-Resume-June2025.docx'),
 (1,
  '/Users/suryan0800/Documents/Surya Developer Profession/Resume/Surya-Resume-June2025.pdf'),
 (2,
  '/Users/suryan0800/Documents/Surya Developer Profession/Resume/0027BY744_Surya_Developer Profession.docx'),
 (3,
  '/Users/suryan0800/Documents/Surya Developer Profession/Education/M.Tech PROVISIONAL CERTIFICATE & Transcript Surya.pdf'),
 (4,
  '/Users/suryan0800/Documents/Surya Developer Profession/Education/M.Tech Semester 3 Internal Marks.xlsx'),
 (5,
  '/Users/suryan0800/Documents/Surya Developer Profession/Education/M.Tech Semester 2 Internal Marks.xlsx'),
 (6,
  '/Users/suryan0800/Documents/Surya Developer Profession/Education/M.Tech Semester 1 Internal Marks.xlsx'),
 (7,
  '/Users/suryan0800/Documents/Surya Developer Profession/Education/M.Tech Semester 4 Internal Marks.xlsx'),
 (8,
  '/Users/suryan0800/Documents/Surya Developer Profession/Jira/Zenhub_CostSqueezers_Cost_Optimi

In [9]:
documents[4]

Document(metadata={'source': '/Users/suryan0800/Documents/Surya Developer Profession/Education/M.Tech Semester 3 Internal Marks.xlsx', 'doc_type': 'Education'}, page_content='Subject Exam Type Max Marks Obtain Marks Subject Exam Type Max Marks Obtain Marks Overall Grade DATA STRUCTURES AND ALGO DESIGN EC 2R 10 4 DATA STRUCTURES AND ALGO DESIGN Total 100 61 B DATA STRUCTURES AND ALGO DESIGN EC 2R 5 3 DISTRIBUTED COMPUTING Total 100 74.05 B- DATA STRUCTURES AND ALGO DESIGN EC 2R 3 2 DATABASE DESIGN AND APPLICATIONS Total 100 78.5 B DATA STRUCTURES AND ALGO DESIGN EC 2R 5 2 CLOUD COMPUTING Total 100 71.75 B- DATA STRUCTURES AND ALGO DESIGN EC 2R 4 2 DATA STRUCTURES AND ALGO DESIGN EC 2R 5 3 DATA STRUCTURES AND ALGO DESIGN EC 2R 3 2 DATA STRUCTURES AND ALGO DESIGN Quiz 5 4 Max Marks Obtain Marks Max Marks Obtain Marks Max Marks Obtain Marks Max Marks Obtain Marks Grade DATA STRUCTURES AND ALGO DESIGN Quiz 5 4.4 Subject EC 1 EC 1 EC 2R EC 2R EC 3R EC 3R EC 1 + EC 2R + EC 3R EC 1 + EC 2R + E

## A sidenote on Embeddings, and "Auto-Encoding LLMs"

We will be mapping each chunk of text into a Vector that represents the meaning of the text, known as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input.
It's different to all the other LLMs we've discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-encoding LLMs are often used for classification.

### Sidenote

In week 8 we will return to RAG and vector embeddings, and we will use an open-source vector encoder so that the data never leaves our computer - that's an important consideration when building enterprise systems and the data needs to remain internal.

In [10]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

# embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

/var/folders/y1/6qt3w0bj299dpqtstwkc1nwc0000gn/T/ipykernel_58961/2593190036.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)


Vectorstore created with 98 documents


In [11]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 98 vectors with 768 dimensions in the vector store


## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [12]:
# Prework (with thanks to Jon R for identifying and fixing a bug in this!)

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange', 'yellow', 'violet'][['Education', 'Jira', 'Stats and Diagrams', 'Eminence', 'Resume', 'Awards'].index(t)] for t in doc_types]

In [13]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
OMP: Warning #96: Cannot form a team with 10 threads, using 1 instead.
OMP: Hint Consider unsetting KMP_DEVICE_THREAD_LIMIT (KMP_ALL_THREADS), KMP_TEAMS_THREAD_LIMIT, and OMP_THREAD_LIMIT (if any are set).


In [14]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Time to use LangChain to bring it all together

In [15]:
# create a new Chat with OpenAI
# llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Alternative - if you'd like to use Ollama locally, uncomment this line instead
llm = ChatOpenAI(temperature=0.7, model_name=GRANITE_MODEL, base_url='http://localhost:11434/v1', api_key='ollama')

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

OWNER = 'Mr. Surya Narayanan'

# Define your system instruction
system_instruction = """You are a Personal Assistant of Mr. Surya Narayanan.  Use the following knowledge context of Surya Narayanan to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer."""

# Answer question
qa_system_prompt = (
    f"You are an Personal assistant of {OWNER} for question-answering tasks about himself.\n"
    f"Use the following retrieved context of {OWNER} to answer the question about himself. "
    f"Always tell positive things about {OWNER}. "
    "If you don't know the answer, just say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n\n"
    "Context: {context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("user", "{input}"),
    ]
)

combine_docs_chain = create_stuff_documents_chain(
    llm, qa_prompt
)


# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = create_retrieval_chain(retriever, combine_docs_chain)



/var/folders/y1/6qt3w0bj299dpqtstwkc1nwc0000gn/T/ipykernel_58961/2885823559.py:8: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [16]:
qa_prompt.messages[1].__dict__

{'variable_name': 'chat_history', 'optional': False, 'n_messages': None}

In [17]:
# Let's try a simple question

query = "Who is Surya?"
chat_history = []
result = conversation_chain.invoke({"input": query, 'chat_history': chat_history})
print(result["answer"])

[chain/start] [chain:retrieval_chain] Entering Chain run with input:
{
  "input": "Who is Surya?",
  "chat_history": []
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "Who is Surya?",
  "chat_history": []
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": "Who is Surya?",
  "chat_history": []
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents] Entering Chain run with input:
{
  "input": "Who is Surya?",
  "chat_history": []
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents > chain:RunnableLambda] Entering Chain run with input:
{
  "input": "Who is Surya?",
  "chat_history": []
}
[chain/end] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:Runnable

In [18]:
result

{'input': 'Who is Surya?',
 'chat_history': [],
 'context': [Document(id='63e2d689-f925-4f76-bf28-1477977ef168', metadata={'doc_type': 'Resume', 'source': '/Users/suryan0800/Documents/Surya Developer Profession/Resume/Surya-Resume-June2025.pdf'}, page_content='Surya Narayanan Srinivasan\n\nSoftware Developer | Data Analytics Specialist suryan0800@gmail.com | +91 7397152594 | Bengaluru, India LinkedIn | GitHub | Credly\n\nProfessional Summary\n\nDriven Software Developer with over 3 years of experience at IBM, specializing in cloud-native applications, data analytics pipelines, and machine learning solutions. Skilled at designing, optimizing ETL workflows, automating deployments, and leveraging AI models to drive business outcomes. Proven leader in cross-functional teams, delivering scalable, resilient, and production-ready systems that improve performance and streamline operations. Experienced in JVM-based backend development, Spring Boot, microservices architecture, and Agile delivery

## Now we will bring this up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [19]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"input": question, 'chat_history': history})
    return result["answer"]

In [21]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True, debug=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[chain/start] [chain:retrieval_chain] Entering Chain run with input:
{
  "input": "Prepare a Resume for Surya",
  "chat_history": []
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "Prepare a Resume for Surya",
  "chat_history": []
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": "Prepare a Resume for Surya",
  "chat_history": []
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents] Entering Chain run with input:
{
  "input": "Prepare a Resume for Surya",
  "chat_history": []
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents > chain:RunnableLambda] Entering Chain run with input:
{
  "input": "Prepare a Resume for Surya",
  "chat_history": []
}
[chain/end] [chain

# Exercises

Try applying this to your own folder of data, so that you create a personal knowledge worker, an expert on your own information!